# Trabalho de Engenharia de Dados II

## Membros
* Clairton Carneiro Luz
* Lairton Mendes
* Paulo Roberto Mesquita da Silva

## Instalando dependências

In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Baixando dataset

In [ ]:

!mkdir -p datasets
# if not exist, download the dataset
import os
import gdown

if not os.path.exists('datasets/archive.zip'):
    url = 'https://drive.google.com/uc?id=17lmw3IkAM4T71chEJkxYNQSB-Ky9s14-'
    output = 'datasets/archive.zip'
    gdown.download(url, output, quiet=False)

## Extraindo dataset

In [ ]:
#  if not exist, unzip the dataset
import zipfile
import os

if not os.path.exists('datasets/past-data.csv'):
    with zipfile.ZipFile('datasets/archive.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets/')

## Carregando dataset

In [4]:
# Imprime leganda do dataset
with open('datasets/headers-details.txt', 'r') as file:
    print(file.read())


Key to data:

Season = League Season
Div = League Division
Date = Match Date (dd/mm/yy)
Time = Time of match kick off
HomeTeam = Home Team
AwayTeam = Away Team
FTHG = Full Time Home Team Goals
FTAG = Full Time Away Team Goals
FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
HTHG = Half Time Home Team Goals
HTAG = Half Time Away Team Goals
HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

Match Statistics (where available)
Referee = Match Referee
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
HC = Home Team Corners
AC = Away Team Corners
HF = Home Team Fouls Committed
AF = Away Team Fouls Committed
HY = Home Team Yellow Cards
AY = Away Team Yellow Cards
HR = Home Team Red Cards
AR = Away Team Red Cards


In [1]:
# Carregando = 'datasets/onlinefoods.csv' usando spark que está rodando localmente using componse.yml

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder \
    .appName("Football Data | Top 5 European Leagues") \
    .master("local[2]") \
    .getOrCreate()

sc = spark.sparkContext

df=spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv('./datasets/past-data.csv')

df.show(5)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 16:42:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+--------------+----------+-------------------+--------------+-------------+----+----+---+----+----+---+--------+----+----+----+---+----+----+----+---+---+---+---+---+
|Season|           Div|      Date|               Time|      HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR| Referee|  HS|  AS| HST|AST|  HF|  AF|  HC| AC| HY| AY| HR| AR|
+------+--------------+----------+-------------------+--------------+-------------+----+----+---+----+----+---+--------+----+----+----+---+----+----+----+---+---+---+---+---+
| 22-23|Premier League|05/08/2022|2024-04-02 20:00:00|Crystal Palace|      Arsenal| 0.0| 2.0|  A| 0.0| 1.0|  A|A Taylor|10.0|10.0| 2.0|2.0|16.0|11.0| 3.0|5.0|1.0|2.0|0.0|0.0|
| 22-23|Premier League|06/08/2022|2024-04-02 12:30:00|        Fulham|    Liverpool| 2.0| 2.0|  D| 1.0| 0.0|  H|A Madley| 9.0|11.0| 3.0|4.0| 7.0| 9.0| 4.0|4.0|2.0|0.0|0.0|0.0|
| 22-23|Premier League|06/08/2022|2024-04-02 15:00:00|   Bournemouth|  Aston Villa| 2.0| 0.0|  H| 1.0| 0.0|  H|P Bankes| 7.0|

24/04/02 16:42:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Limpeza e transformação

In [5]:
# traduzindo colunas para pt-br
df = df.withColumnRenamed('Season', 'temporada')
df = df.withColumnRenamed('Div', 'divisao')
df = df.withColumnRenamed('Date', 'data')
df = df.withColumnRenamed('Time', 'inicio_partida')
df = df.withColumnRenamed('HomeTeam', 'time_casa')
df = df.withColumnRenamed('AwayTeam', 'time_fora')
df = df.withColumnRenamed('FTHG', 'gols_time_casa')
df = df.withColumnRenamed('FTAG', 'gols_time_fora')
df = df.withColumnRenamed('FTR', 'resultado')
df = df.withColumnRenamed('HTHG', 'time_casa_gols_intervalo')
df = df.withColumnRenamed('HTAG', 'time_fora_gols_intervalo')
df = df.withColumnRenamed('HTR', 'resultado_intervalo')
df = df.withColumnRenamed('Referee', 'arbitro')
df = df.withColumnRenamed('HS', 'chutes_casa')
df = df.withColumnRenamed('AS', 'chutes_fora')
df = df.withColumnRenamed('HST', 'chutes_no_gol_casa')
df = df.withColumnRenamed('AST', 'chutes_no_gol_fora')
df = df.withColumnRenamed('HC', 'escanteios_casa')
df = df.withColumnRenamed('AC', 'escanteios_fora')
df = df.withColumnRenamed('HF', 'faltas_casa')
df = df.withColumnRenamed('AF', 'faltas_fora')
df = df.withColumnRenamed('HY', 'cartoes_amarelos_casa')
df = df.withColumnRenamed('AY', 'cartoes_amarelos_fora')
df = df.withColumnRenamed('HR', 'cartoes_vermelhos_casa')
df = df.withColumnRenamed('AR', 'cartoes_vermelhos_fora')

df.show(5)


+---------+--------------+----------+-------------------+--------------+-------------+--------------+--------------+---------+------------------------+------------------------+-------------------+--------+-----------+-----------+------------------+------------------+-----------+-----------+---------------+---------------+---------------------+---------------------+----------------------+----------------------+
|temporada|       divisao|      data|     inicio_partida|     time_casa|    time_fora|gols_time_casa|gols_time_fora|resultado|time_casa_gols_intervalo|time_fora_gols_intervalo|resultado_intervalo| arbitro|chutes_casa|chutes_fora|chutes_no_gol_casa|chutes_no_gol_fora|faltas_casa|faltas_fora|escanteios_casa|escanteios_fora|cartoes_amarelos_casa|cartoes_amarelos_fora|cartoes_vermelhos_casa|cartoes_vermelhos_fora|
+---------+--------------+----------+-------------------+--------------+-------------+--------------+--------------+---------+------------------------+---------------------

### Criar colunas de precisões dos chutes

In [6]:
# precisão dos chutes no gol do time de casa e de fora
df = df.withColumn('precisao_chutes_casa', df['chutes_no_gol_casa'] / df['chutes_casa'])
df = df.withColumn('precisao_chutes_fora', df['chutes_no_gol_fora'] / df['chutes_fora'])
df.select('time_casa', 'time_fora', 'chutes_no_gol_casa', 'chutes_casa', 'precisao_chutes_casa', 'chutes_no_gol_fora', 'chutes_fora', 'precisao_chutes_fora').show(5)

+--------------+-------------+------------------+-----------+--------------------+------------------+-----------+--------------------+
|     time_casa|    time_fora|chutes_no_gol_casa|chutes_casa|precisao_chutes_casa|chutes_no_gol_fora|chutes_fora|precisao_chutes_fora|
+--------------+-------------+------------------+-----------+--------------------+------------------+-----------+--------------------+
|Crystal Palace|      Arsenal|               2.0|       10.0|                 0.2|               2.0|       10.0|                 0.2|
|        Fulham|    Liverpool|               3.0|        9.0|  0.3333333333333333|               4.0|       11.0| 0.36363636363636365|
|   Bournemouth|  Aston Villa|               3.0|        7.0| 0.42857142857142855|               2.0|       15.0| 0.13333333333333333|
|         Leeds|       Wolves|               4.0|       12.0|  0.3333333333333333|               6.0|       15.0|                 0.4|
|     Newcastle|Nott'm Forest|              10.0|      